Інсталяція і імпорт бібліотек

In [ ]:
!pip install pandas numpy scikit-learn tensorflow

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

Завантаження даних і препроцесинг

In [8]:
data = pd.read_csv("Churn_Modelling.csv")

In [9]:
# Видалення зайвих стовпців
data_cleaned = data.drop(columns=["RowNumber", "CustomerId", "Surname"])

# Перетворення категоріальних змінних у числові
data_encoded = pd.get_dummies(data_cleaned, columns=["Geography", "Gender"], drop_first=True)

# Нормалізація числових змінних
scaler = MinMaxScaler()
numerical_columns = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]
data_encoded[numerical_columns] = scaler.fit_transform(data_encoded[numerical_columns])

# Розділення даних на ознаки (X) та цільову змінну (y)
X = data_encoded.drop(columns=["Exited"])
y = data_encoded["Exited"]

Кореляційний аналіз

In [10]:
correlation_matrix = data_encoded.corr()
print("Кореляція із залежною змінною (Exited):")
print(correlation_matrix["Exited"].sort_values(ascending=False))

Кореляція із залежною змінною (Exited):
Exited               1.000000
Age                  0.285323
Geography_Germany    0.173488
Balance              0.118533
EstimatedSalary      0.012097
HasCrCard           -0.007138
Tenure              -0.014001
CreditScore         -0.027094
NumOfProducts       -0.047820
Geography_Spain     -0.052667
Gender_Male         -0.106512
IsActiveMember      -0.156128
Name: Exited, dtype: float64


Крос-валідація

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for train_index, val_index in skf.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    y_pred = (model.predict(X_val) > 0.5).astype("int32")
    fold_results.append({
        "classification_report": classification_report(y_val, y_pred, output_dict=True),
        "confusion_matrix": confusion_matrix(y_val, y_pred).tolist(),
        "roc_auc": roc_auc_score(y_val, model.predict(X_val))
    })

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step


Оцінка метрики якості

In [12]:
mean_accuracy = np.mean([result["classification_report"]["accuracy"] for result in fold_results])
mean_roc_auc = np.mean([result["roc_auc"] for result in fold_results])

print(f"Середня точність (Accuracy): {mean_accuracy}")
print(f"Середній ROC AUC: {mean_roc_auc}")

Середня точність (Accuracy): 0.8543
Середній ROC AUC: 0.8529069199308636
